In [26]:
from module.setting import instCpStockCode, instCpCodeMgr, instStockChart, instCpCybos
from module import connection, get, save, search, update
import pandas as pd
import os
import time
from module.setting import instCpCybos
from module.get import get_stock_info
from module.search import search_by_code
from module.update import update_stock_list
from random import *
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta
import pandas as pd
import os
import time

In [34]:
def stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=1)
    
    stock_df = get_stock_info(stock_code, end_day, end_day, type) # 첫번째 DF 생성 => pre_day '20221028' end_day '20221103'
    
    stock_name = search_by_code(stock_code)

    for day in range(6): # range 범위 수정하지 말 것
        transfer_pre_day = datetime.strptime(end_day, day_format) # '20221028' '20221023'
        pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format) # '20221023' '20221018'

        info_second = get_stock_info(stock_code, pre_day, pre_day, type) # '20221023' '20221027'

        time.sleep(uniform(0.3,0.5))
        end_day = pre_day
        print(pre_day)

        stock_df = pd.concat([stock_df, info_second], axis=0)
        info_second = pd.DataFrame()
        print(stock_df.shape)
        print(stock_name, 'Count : {0}, 남은 요청 횟수 :{1}'.format(day, instCpCybos.GetLimitRemainCount(1)))
        
        if  instCpCybos.GetLimitRemainCount(1) < 5:
            time.sleep(10)
    
    # stock_df.to_csv(r'\\DESKTOP-H2H6JNB\data\data\{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

In [35]:
stock_info = stock_info_auto('A066970', '20221128', 'm')

20221127
(381, 25)
['A066970', '엘앤에프'] Count : 0, 남은 요청 횟수 :58
20221126
(381, 25)
['A066970', '엘앤에프'] Count : 1, 남은 요청 횟수 :57
20221125
(762, 25)
['A066970', '엘앤에프'] Count : 2, 남은 요청 횟수 :56
20221124
(1143, 25)
['A066970', '엘앤에프'] Count : 3, 남은 요청 횟수 :55
20221123
(1524, 25)
['A066970', '엘앤에프'] Count : 4, 남은 요청 횟수 :54
20221122
(1905, 25)
['A066970', '엘앤에프'] Count : 5, 남은 요청 횟수 :53


In [37]:
stock_info.to_csv('./test_A066970.csv', encoding='utf-8-sig')

In [44]:
def day_stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=5) # 5일치
    
    stock_df = pd.DataFrame()
    
    stock_name = search_by_code(stock_code)

    for day in range(2): # range 범위 수정하지 말 것
        transfer_pre_day = datetime.strptime(end_day, day_format) # '20221028' '20221023'
        pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format) # '20221023' '20221018'

        info_second = get_stock_info(stock_code, pre_day, end_day, type) # '20221023' '20221027'

        time.sleep(0.15)
        end_day = pre_day
        print(pre_day)

        stock_df = pd.concat([stock_df, info_second], axis=0)
        info_second = pd.DataFrame()
        print(stock_df.shape)
        print(stock_name, 'Count : {0}, 남은 요청 횟수 :{1}'.format(day, instCpCybos.GetLimitRemainCount(1)))
        
        if  instCpCybos.GetLimitRemainCount(1) < 5:
            time.sleep(10)
    
    # stock_df.to_csv(r'\\DESKTOP-H2H6JNB\data\day_data\day_{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

In [47]:
day_stock_info = stock_info_auto('A066970', '20221128', 'D')

20221127
(1, 25)
['A066970', '엘앤에프'] Count : 0, 남은 요청 횟수 :58
20221126
(1, 25)
['A066970', '엘앤에프'] Count : 1, 남은 요청 횟수 :57
20221125
(2, 25)
['A066970', '엘앤에프'] Count : 2, 남은 요청 횟수 :56
20221124
(3, 25)
['A066970', '엘앤에프'] Count : 3, 남은 요청 횟수 :55
20221123
(4, 25)
['A066970', '엘앤에프'] Count : 4, 남은 요청 횟수 :54
20221122
(5, 25)
['A066970', '엘앤에프'] Count : 5, 남은 요청 횟수 :53


In [49]:
day_stock_info.to_csv('./test_day_A066970.csv', encoding='utf-8-sig')

In [50]:
def concat(): 
    
    stock_min_df = pd.read_csv('./test_A066970.csv', index_col='Unnamed: 0')
    stock_day_df = pd.read_csv('./test_day_A066970.csv', index_col='Unnamed: 0')
    

    for v in stock_min_df.index:
        input_list = ['전일대비','상장주식수','시가총액','외국인주문한도수량'
                ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
                ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량']
        
        stock_min_df.loc[v, input_list] = stock_day_df.loc[v, input_list].values
        stock_min_df.to_csv('./test_concat_day_A066970.csv', encoding='utf-8-sig') 

In [51]:
concat()

In [58]:
def label():

    stock_info = pd.read_csv('./test_concat_day_A066970.csv', index_col=0)
    
    # 데이터 프레임에서 날짜 인덱스 추출
    date = stock_info.index.unique()

    # label 컬럼 추가 후 0으로 초기화
    stock_info['label'] = 0

    # 업데이트에 사용할 데이터 프레임 생성
    update_stock_info = pd.DataFrame()

    # labeling
    for day in date:

        # 특정일의 Data 추출
        select_day = stock_info.loc[day].copy()
        select_day['label'] = 0
        
        # 특정일의 Row 만큼 반복
        for row in range(len(select_day)):
            
            # 특정일의 현재 row 이후 최대 고가를 추출
            next_price = select_day[-row-1::-1]['고가'].max()

            # 추출한 최대 고가를 label 컬럼에 대입
            select_day.iloc[-row-1,-1] = next_price
            next_price = 0
        
        # 특정일 label이 추가된 DF를 업데이트할 DF에 concat
        update_stock_info = pd.concat([update_stock_info, select_day])

    trans = update_stock_info.loc[:,['고가','label']]
    trans = trans.rename(columns={'label':'pct_label'}).T

    get_trans = trans.pct_change().T.iloc[:,-1]
    
    update_stock_info = pd.concat([update_stock_info, get_trans], axis=1)
    update_stock_info = update_stock_info.drop(['label'], axis=1)

    update_stock_info['pct_label'] = update_stock_info['pct_label'].mul(100)
    update_stock_info['pct_label'] = update_stock_info['pct_label'].round(1)

    update_stock_info.drop(columns=['등락주선','등락비율','예탁금','주식회전율','거래성립률'],axis=1, inplace=True)

    update_stock_info.to_csv('./test_complete_A066970.csv', encoding='utf-8-sig')

In [59]:
label()